# Online Sales Data Visualization

## Intro

The goal of this portion of the project is to complete exploritory analysis and then preview the graphs before implementing them into the dash app file: `sales_dash_app.py`. 

The libraries used in both parts are:
* Plotly: https://plotly.com/python/
* Dash: (Also Plotly Basically): https://dash.plotly.com/tutorial
* Pandas: https://pandas.pydata.org/docs/

The dataset was found on Kaggle. Link to data: https://www.kaggle.com/datasets/shreyanshverma27/online-sales-dataset-popular-marketplace-data

## Import Libraries

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Verify the data is clean

In [2]:
# Read Data
data = pd.read_csv('Online Sales Data.csv')

# Visualize
data.head()

,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue,Region,Payment Method
0,10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card
1,10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal
2,10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card
3,10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card
4,10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal


In [3]:
# Check for any null values
data.isnull().sum()

Transaction ID      0
Date                0
Product Category    0
Product Name        0
Units Sold          0
Unit Price          0
Total Revenue       0
Region              0
Payment Method      0
dtype: int64

In [4]:
# Check for any duplicates
data.duplicated().sum()

0

In [5]:
# See what values are in the categorical column
data['Region'].unique()

array(['North America', 'Europe', 'Asia'], dtype=object)

In [6]:
# See what values are in the categorical column
data['Payment Method'].unique()

array(['Credit Card', 'PayPal', 'Debit Card'], dtype=object)

In [7]:
# See what values are in the categorical column
unique_Product_name = data['Product Name'].unique()

# Column has to many to print
len(unique_Product_name)

232

In [8]:
# See what values are in the categorical column
unique_Product_cat = data['Product Category'].unique()
len(unique_Product_cat)

6

In [9]:
unique_Product_cat

array(['Electronics', 'Home Appliances', 'Clothing', 'Books',
       'Beauty Products', 'Sports'], dtype=object)

## Prep data for Charts and Chart scripts

In [10]:
# Convert Columns to pandas dataframes and join them into their own frames
Dates = data['Date']
Dates = pd.to_datetime(Dates)
total_Rev = data['Total Revenue']
Dates_df = Dates.to_frame(name='Dates_df')
Total_rev_df = total_Rev.to_frame(name='Total_rev_df')
sales_by_month_data = pd.merge(Dates_df, Total_rev_df, left_index=True, right_index=True, how='outer')

# Set the Date column as the index
sales_by_month_data.set_index('Dates_df', inplace=True)

# Sort the totals by month and calculate the sum
sales_by_month = sales_by_month_data.resample('M').sum()

# Verify it work
sales_by_month.head()

C:\Users\sdelo\AppData\Local\Temp\ipykernel_26876\3684046095.py:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  sales_by_month = sales_by_month_data.resample('M').sum()


,Total_rev_df
Dates_df,
2024-01-31,14548.32
2024-02-29,10803.37
2024-03-31,12849.24
2024-04-30,12451.69
2024-05-31,8455.49


In [11]:
sales_by_month.shape

(8, 1)

In [12]:
# Print Chart
fig = px.bar(sales_by_month, x=sales_by_month.index, y='Total_rev_df', title='Total Sales by Month')
fig.show()

In [13]:
# Merge them by date
sales_by_date_data = pd.merge(Dates_df, Total_rev_df, left_index=True, right_index=True, how='outer')
# Set the Date column as the index
sales_by_date_data.set_index('Dates_df', inplace=True)
sales_by_date = sales_by_date_data.resample('D').sum()
sales_by_date.head()

,Total_rev_df
Dates_df,
2024-01-01,1999.98
2024-01-02,499.99
2024-01-03,209.97
2024-01-04,63.96
2024-01-05,89.99


In [14]:
# Print this one but include a range slider
fig = px.line(sales_by_date, x=sales_by_date.index, y='Total_rev_df', title='Total Sales by Date')

# range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

In [15]:
# Make categories into a df
categories = data['Product Category']
categories_df = categories.to_frame(name='Categories')

# Merge it with revenue and sum them after grouping
total_Rev_by_category = pd.merge(categories_df, Total_rev_df, left_index=True, right_index=True, how='outer')
total_Rev_by_category_grouped = total_Rev_by_category.groupby('Categories')['Total_rev_df'].sum().reset_index()
total_Rev_by_category_grouped

,Categories,Total_rev_df
0,Beauty Products,2621.90
1,Books,1861.93
2,Clothing,8128.93
3,Electronics,34982.41
4,Home Appliances,18646.16
5,Sports,14326.52


In [16]:
# Chart
fig = px.bar(total_Rev_by_category_grouped, x=total_Rev_by_category_grouped['Categories'], y=total_Rev_by_category_grouped['Total_rev_df'], title='Revenue grouped by Category')
fig.show()

In [17]:
# Convert regions to a df
regions = data['Region']
regions_df = regions.to_frame(name='Region')

# Join it with revenue 
Rev_by_region = pd.merge(regions_df, Total_rev_df, left_index=True, right_index=True, how='outer')

# Join that with date
Rev_by_region_dated = pd.merge(Rev_by_region, Dates_df, left_index=True, right_index=True, how='outer')
Rev_by_region_dated.set_index('Dates_df', inplace=True)

# Group by region
Rev_by_region_dated_grouped = Rev_by_region_dated.groupby('Region')
Rev_by_region_dated_grouped.head()

,Region,Total_rev_df
Dates_df,,
2024-01-01,North America,1999.98
2024-01-02,Europe,499.99
2024-01-03,Asia,209.97
2024-01-04,North America,63.96
2024-01-05,Europe,89.99
2024-01-06,Asia,149.95
2024-01-07,North America,2499.99
2024-01-08,Europe,1199.98
2024-01-09,Asia,539.94


In [18]:
# get data for each group
North_America = Rev_by_region_dated_grouped.get_group('North America')
North_America

,Region,Total_rev_df
Dates_df,,
2024-01-01,North America,1999.98
2024-01-04,North America,63.96
2024-01-07,North America,2499.99
2024-01-10,North America,51.98
2024-01-13,North America,1499.98
...,...,...
2024-08-13,North America,80.97
2024-08-16,North America,169.00
2024-08-19,North America,19.98


In [19]:
Europe = Rev_by_region_dated_grouped.get_group('Europe')
Asia = Rev_by_region_dated_grouped.get_group('Asia')

In [20]:
# Plot chart
fig = px.line(North_America, x=North_America.index, y='Total_rev_df', title='Total Sales by Date')

# Add line for Europe
fig.add_trace(go.Scatter(x=Europe.index, y=Europe['Total_rev_df'],
                    mode='lines',
                    name='Europe Data'))

# Add line for Asia
fig.add_trace(go.Scatter(x=Asia.index, y=Asia['Total_rev_df'],
                    mode='lines',
                    name='Asia Data'))

# range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

In [21]:
fig = px.pie(data, values='Total Revenue', names='Payment Method', title='Percentage of Sales by Payment Method')
fig.show()